### PHYS2641/3681 Laboratory Skills & Electronics

# Electronics Laboratory Practical 4: Modulation



### Jupyter Notebook Instructions


**When you have downloaded this Notebook, you MUST**

- **Click the button at the top right of the screen labelled 'Not Trusted', and select 'Trust'**

- **Select 'Cell -> Run All' from the Jupyter menu bar**
    
**If you fail to do these the Notebook will not work.**


The data that you enter into the Notebooks is saved within a '.pkl' ('pickle') file on the desktop of your computer. This is updated every time you enter data but ensure, at the end of the session, that you press the 'save' button at the bottom of the notebook at the end of the session to ensure all data/inputs are saved, then exit the notebook by using 'File -> Close and Halt' from the Jupyter menu bar.

Your laboratory pairing can then each take a copy of the '.pkl' file from the desktop - which contains the data etc.that you have entered. You can re-load this data on your own computer by placing the '.pkl' file in the same directory as the downloaded copy of this Notebook - note that a new '.pkl' file  with any further changes will be saved to your desktop, and the original '.pkl' file that you loaded will not be altered.


**Ensure that both students in your team have a working copy of the completed Jupyter Notebook contents, and any sketches or notes required. Any remaining datafiles will be deleted from the labs laptop at the end of the session.**


### Laboratory Notebook Preperation

Read through this script carefully in advance of the laboratory session. 

You will be recording the results of your experiments partly using the Jupyter Notebook, but you may also wish to take photographs of the circuits that you build, paper notes, sketches and rough work - so bring a notebook if you wish to do so. 

After completing each identified part of the session you should ensure that you fully understand the work that you have done, before moving on to the next part. Ask the staff member in your laboratory to verify the correct operation of your circuit if you are not absolutely certain of the expected behaviour.

---
---

In [1]:
from IPython.display import HTML
HTML('''<script>
$.each(IPython.notebook.get_cells(), function(index, cell){
    if (cell.cell_type !== 'code'){
        $('div.cell').unbind('click');
        $('div.cell').unbind('dblclick');
    }  
});
</script>''')
# Javascript code above unbinds click events for markdown cells to disable editing

In [2]:
# Local version needs to hide code as this cannot be done using the hide_input tags.
from IPython.display import HTML
HTML('''<script>$('div.input').hide()</script>''')

In [3]:
%matplotlib inline
from ipywidgets import widgets, interact, interactive, fixed, Layout
import IPython.display as dsply
from IPython.display import clear_output
import matplotlib.pyplot as pyplot
import numpy

In [4]:
import warnings

class dataset():
    def __init__(self, parent, datastr, colheadings):
        self.parent = parent
        self.colheadings = colheadings
        self.Data = datastr
        
        if self.colheadings == None:
            print(colheadings)
            self.colheadings = [r'$x$', r'$y$']
        
        self.LoadData()
                    
        # initialise the button for adding a line to the table 
        self.AddLineButton = widgets.Button(description='Add line to table')
        self.AddLineButton.on_click(self.AddLineToTable)
        # and removing one 
        self.RemoveLineButton = widgets.Button(description='Remove last line')
        self.RemoveLineButton.on_click(self.RemoveLastLineFromTable)
           
        self.MakeTable()
 
    def MakeTable(self):
        # Creates table made of a grid of textboxes which only accept floats for entering Vin, Vout and R1 and R2 data
        self.Cols = []
        self.ColHs = []
        # creates the table with headings 
        for j, col in enumerate(self.colheadings):
            widgetlist=[widgets.FloatText(self.Data[i][j]) for i in range(self.DataPoints)]
            for widget in widgetlist:
                widget.observe(self.table_widget_on_change, names='value')
            self.Cols.append(widgets.VBox(widgetlist))
            self.ColHs.append(widgets.VBox([widgets.Label(value=self.colheadings[j]), self.Cols[j]]))           
    
        self.Table = widgets.HBox(self.ColHs)
        
        dsply.display(self.RemoveLineButton)
        dsply.display(self.Table)
        dsply.display(self.AddLineButton)
        
    def table_widget_on_change(self, change):
        self.Save()
    
    def AddLineToTable(self, btn):
        # function stores the data entered into the text boxes in the table then clears the table 
        for j in range(len(self.colheadings)):
            for i in range(self.DataPoints):
                self.Data[i][j] = self.Cols[j].children[i].value
        
        clear_output()#clear the Jupyter output
        self.Table.close() # closes the current table 
        self.DataPoints += 1 #add one to datapoints
        self.Data.append([0.0 for i in range(len(self.colheadings))])
        self.MakeTable()
                        
    def RemoveLastLineFromTable(self, btn):
        # function stores the data entered into the text boxes in the table then clears the table 
        clear_output()#clear the Jupyter output
        self.Table.close() # closes the current table 
        if self.DataPoints > 2:
            self.DataPoints -= 1 #subtract one to datapoints
            del(self.Data[-1])       
        self.MakeTable()
         
    def Save(self):
        for j in range(len(self.colheadings)):
            for i in range(self.DataPoints):
                self.Cols[j].children[i].disabled=True
                self.Data[i][j] = self.Cols[j].children[i].value               
        self.parent.SaveData()
        for j in range(len(self.colheadings)):
            for i in range(self.DataPoints):
                self.Cols[j].children[i].disabled=False          
    
    def LoadData(self):
        if len(self.Data)==0:       #initial number of datapoints/rows in the table 
            self.DataPoints = 3   # initialise data list for storing the values from the table text boxes
            for j in range(self.DataPoints):
                self.Data.append([0.0 for i in range(len(self.colheadings))]) 
        else:
            self.DataPoints = len(self.Data)
            
    



class bodedataset(dataset):
    def __init__(self, parent, datastr, colheadings):
        self.VPrecBox = widgets.HBox([widgets.Label(value='Precision of V measurement (mV):'), widgets.FloatText(value=1.0)])
        self.GenerateButton = widgets.Button(description='Generate Bode Plots')
        self.GenerateButton.on_click(self.PlotData)  
        
        super().__init__(parent, datastr, colheadings)
        
        clear_output()#clear the Jupyter output
        self.Table.close() # closes the current table         
        self.VPrecText = widgets.FloatText(value=self.VPrecision)
        self.VPrecBox = widgets.HBox([widgets.Label(value='Precision of V measurement (mV):'), self.VPrecText])
        #bind widget change to save data
        self.VPrecText.observe(self.prec_widget_on_change, names='value')

        self.MakeTable()
        
    def prec_widget_on_change(self, change):
        self.Save()
    
    def LoadData(self):
        if len(self.Data)==0:       #initial number of datapoints/rows in the table 
            self.DataPoints = 4   # initialise data list for storing the values from the table text boxes
            for j in range(self.DataPoints+1):
                self.Data.append([0.0 for i in range(len(self.colheadings))]) 
            #self.VPrecision = 0.0
        #else:
        self.VPrecision = self.Data[-1][0]
        self.DataPoints = len(self.Data)-1
        
        self.Frequency = numpy.array([self.Data[i][0] for i in range(self.DataPoints)])
        self.FrequencyErr = numpy.array([self.Data[i][1] for i in range(self.DataPoints)])
        self.Gain = numpy.array([0.0 for i in range(self.DataPoints)])
        self.GainErr = numpy.array([0.0 for i in range(self.DataPoints)])
                
    def Save(self):
        self.VPrecision = self.VPrecText.value
        self.Data[-1][0] = self.VPrecision
        super().Save()
        
    def AddLineToTable(self, btn):
        # function stores the data entered into the text boxes in the table then clears the table 
        for j in range(len(self.colheadings)):
            for i in range(self.DataPoints):
                self.Data[i][j] = self.Cols[j].children[i].value
        
        clear_output()#clear the Jupyter output
        self.Table.close() # closes the current table 
        self.DataPoints += 1 #add one to datapoints
        self.Data.append([0.0 for i in range(len(self.colheadings))])
        self.Data[-1][0] = self.VPrecision
        self.Data[-2][0] = 0.0
        self.MakeTable()
        
    def RemoveLastLineFromTable(self, btn):
        # function stores the data entered into the text boxes in the table then clears the table 
        clear_output()#clear the Jupyter output
        self.Table.close() # closes the current table 
        if self.DataPoints > 2:
            self.DataPoints -= 1 #subtract one to datapoints
            del(self.Data[-1])
        self.Data[-1] = [0.0 for i in range(len(self.colheadings))]
        self.Data[-1][0] = self.VPrecision
        self.MakeTable()
        
    def MakeTable(self):
        super().MakeTable()
        dsply.display(self.VPrecBox)
                
    def PlotButton(self):
        dsply.display(self.GenerateButton)
        
    def PlotData(self, btn):        
        clear_output()
        dsply.display(self.GenerateButton)
        
        self.Frequency = numpy.array([self.Data[i][0] for i in range(self.DataPoints)])
        self.FrequencyErr = numpy.array([self.Data[i][1] for i in range(self.DataPoints)])
        VinData = numpy.array([self.Data[i][2] for i in range(self.DataPoints)])
        VoutData = numpy.array([self.Data[i][3] for i in range(self.DataPoints)])
        #TSData = numpy.array([self.Data[i][4] for i in range(self.DataPoints)])
        #TEData = numpy.array([self.Data[i][5] for i in range(self.DataPoints)])
        
        #calculate gain and phase and their errors
        with warnings.catch_warnings():#This prevents divide by zero warnings showing when calculating phase error
            warnings.simplefilter("ignore")
            self.Gain = 20*numpy.log10((VoutData/VinData))
            #self.Phase = TSData*0.000001*360*self.Frequency
            #self.GainErr = self.Gain*((self.VPrecision/VinData)**2+(self.VPrecision/VoutData)**2)**0.5
            self.GainErr = 20*(((self.VPrecision/VinData)**2+(self.VPrecision/VoutData)**2)**0.5)/(numpy.log(10)*(VoutData/VinData))
            #dB_gain_err = 20*Voltage_gain_err/(numpy.log(10)*Voltage_gain) # See inside front cover of Hughes & Hase! 
            #self.PhaseErr = self.Phase*((self.FrequencyErr/self.Frequency)**2+(TEData/TSData)**2)**0.5
        
    
        #plot the bode plots with error bars and log scales for the frequency axis 
        pyplot.figure(figsize = (10,10))
        pyplot.subplot(211)
        pyplot.xscale('log')#, basex=10)  
        pyplot.errorbar(self.Frequency, self.Gain, self.GainErr, ecolor = "blue", color = "blue", marker="x", linestyle="none")
        pyplot.title("Gain Bode Plot")
        pyplot.xlabel("Frequency (Hz)")
        pyplot.ylabel("Gain (dB)")

        #pyplot.subplot(212)
        #pyplot.xscale('log', basex=10)
        #pyplot.errorbar(self.Frequency, self.Phase, self.PhaseErr, ecolor = "blue", color = "blue", marker="x", linestyle="none")
        #pyplot.title("Phase Shift Bode Plot")
        #pyplot.xlabel("Frequency (Hz)")
        #pyplot.ylabel("Phase Shift (°)")     
        
        self.DataPrintButton = widgets.Button(description='Print gain data', layout=Layout(width='200px'))
        self.DataPrintButton.on_click(self.PrintData)
        dsply.display(self.DataPrintButton)
        
    def PrintData(self, btn):
        # prints the gain and phase data for inspection 
        print('Frequency data (Hz):', repr(self.Frequency))
        print('Frequency error data (Hz):', repr(self.FrequencyErr))
        print('Gain data (dB):', repr(self.Gain))
        print('Gain error data (dB):', repr(self.GainErr))
        #print('Phase-shift data (°):', repr(self.Phase))
        #print('Phase-shift error data (°):', repr(self.PhaseErr))
        

           

In [5]:
#Waveform tool
class waveform():
    def __init__(self, parent, datastr):
        self.parent=parent
        self.Data=datastr
        self.widgets=[]
        
        if len(self.Data)==0:
            self.Data.extend(['Not Answered', #waveform
                              0.0, #Freq
                              0.0, #Ampl
                              0.0, #Offs
                              1.0, #t_range
                              1.0])#V_range
    
    
        #initialise the textboxes, buttons and sliders making up the user interface 
        self.widgets.append(widgets.RadioButtons(options=['Sine', 'Square', 'Triangle', 'Sawtooth', 'Not Answered'],
                                                   value=self.Data[0], layout=Layout(height='80px')))
        self.WButtonBox = widgets.HBox([widgets.Label(value='Type of Waveform:'), self.widgets[0]])

        self.widgets.append(widgets.FloatText(value=self.Data[1]))
        self.WFLayout = widgets.HBox([widgets.Label(value='Frequency (kHz):'), self.widgets[1]])

        self.widgets.append(widgets.FloatText(value=self.Data[2]))
        self.WALayout = widgets.HBox([widgets.Label(value='Amplitude (V):'), self.widgets[2]])

        self.widgets.append(widgets.FloatText(value=self.Data[3]))
        self.OBLayout = widgets.HBox([widgets.Label(value='DC Offset (V):'), self.widgets[3]])
                           
        self.widgets.append(widgets.FloatSlider(value=self.Data[4], max=20.0))
        self.AxisLay = widgets.HBox([widgets.Label(value='Time range (ms):'), self.widgets[4]])

        self.widgets.append(widgets.FloatSlider(value=self.Data[5], max=30.0))
        self.YAxisLay = widgets.HBox([widgets.Label(value='Voltage range (V):'), self.widgets[5]])

        # displays user interface elements 
        dsply.display(widgets.VBox([self.WButtonBox, self.WFLayout, self.WALayout, self.OBLayout, self.AxisLay, self.YAxisLay]))
    
    
    def WaveGenButton(self):
        # displays the wave generator button
        self.WaveGenButton = widgets.Button(description='Generate Waveform')
        self.WaveGenButton.on_click(self.WaveGen)
        dsply.display(self.WaveGenButton)
    
    
    def WaveGen(self, btn):    
        # Function which saves the data and generates the waveforms to be displayed
        
        clear_output()
        dsply.display(self.WaveGenButton)
        
        # defines x axis range based on the value chosen on the slider by the user 
        SampleRate = 44100 
        RunTime = self.widgets[4].value #self.AxisSlider.value 
        NSamples = int(SampleRate * RunTime)
        timebase = numpy.linspace(-1*RunTime, RunTime, NSamples)
    
        # error catching if no frequency or amplitude is input 
        if self.widgets[1].value == 0 or self.widgets[2].value == 0:
            print('Please enter a non-zero value for frequency and amplitude')
            return

        # saves data from text boxes, and converts freq to period etc.
        for i in range(len(self.widgets)):
            self.Data[i] = self.widgets[i].value
        self.parent.SaveData()
        
        Frequency = self.widgets[1].value
        Period = 1.0/Frequency
        Amp = self.widgets[2].value
        Offset = self.widgets[3].value

        V = numpy.zeros(NSamples)  # initialise y data array 

        # calculates V function based on data waveform chosen on the radio buttons, with error catching for no data being chosen 
        if self.widgets[0].value == 'Sine': 
            V = Amp*numpy.sin(2*numpy.pi*Frequency*timebase)
        elif self.widgets[0].value == 'Square':
            V = numpy.sin(2*numpy.pi*Frequency*timebase)
            V = Amp*numpy.sign(V)
        elif self.widgets[0].value == 'Triangle':
            V =((2*Amp)/numpy.pi)*numpy.arcsin(numpy.sin((2*numpy.pi*timebase/Period)))
        elif self.widgets[0].value == 'Sawtooth':
            V =-1*((2*Amp)/numpy.pi)*numpy.arctan(1/numpy.tan((numpy.pi*timebase/Period)))
        elif self.widgets[0].value == 'Not Answered':
            print('No waveform type selected:(')
            return
        else:
            print('No waveform type selected :(')
            return

        V = V + Offset # adds the DC offset to the waveform 

        Baseline = numpy.zeros(NSamples) # array used to plot a baseline to make visualising the waveform easier on the eye 

        # plots waveform using matplotlib.pyplot, defining y axis range based on the slider value 
        pyplot.plot(timebase,V)
        pyplot.plot(timebase, Baseline, linestyle = 'dashed', color = 'gray')
        pyplot.xlim(xmin=(-1*self.widgets[4].value), xmax=self.widgets[4].value)

        pyplot.xlabel("Timebase (ms)")
        pyplot.ylabel("Amplitude (V)")
        pyplot.ylim(ymin=(-1*self.widgets[5].value), ymax=self.widgets[5].value)

   

In [6]:
import pickle
import os
from pathlib import Path

class dataclass():
    pass

class Electronics_session4():
    def __init__(self, file):
        self.file = str(Path.home() / 'Desktop' / file) # in desktop
        self.tempfile = '{}.tmp'.format(file)           # in current working directory
        self.widgets=[]
                
        self.SaveButton = widgets.Button(description='Save all data')
        self.SaveButton.on_click(self.Save_pressed)  
         
        try: #Load in the data if it is there - this allows data persistence between students/runs of the notebook.         
             # first look in current directory:
            with open(str(Path(self.file).name), 'rb') as infile:
                self.datastructure = pickle.load(infile)
                loaded = True      
        except OSError: 
            loaded = False
                
        if not loaded:
            try: # now look on Desktop
                with open(self.file, 'rb') as infile:
                    self.datastructure = pickle.load(infile)
                    
            except OSError: # default data 
                self.datastructure = dataclass()
                self.datastructure.wfmdata1 = []
                self.datastructure.wfmdata2 = []
                self.datastructure.lpdata1 = []

                self.datastructure.widgetvals = ['Explain the form of the output signal:',
                                                 'Explain the form of the output signal:',
                                                 'Explain the form of the output signal:',
                                                 'Explain the form of the output signal:',
                                                 'Explain the influence of the DC voltage on the comparator output:',
                                                 'Describe and explain the observed output:',
                                                 'State and justify corner frequency value:',
                                                 'Describe and explain the demodulated signal obtained:',
                                                 'Explain what happens if the modulation frequency is too low:',
                                                 'Explain the behaviour of the LED that you see:']

        #Write saved or default value to widgets:
        self.widgets.append(widgets.Textarea(value=self.datastructure.widgetvals[0], continuous_update=False, layout=Layout(width = '600px', height = '100px')))
        self.widgets.append(widgets.Textarea(value=self.datastructure.widgetvals[1], continuous_update=False, layout=Layout(width = '600px', height = '100px')))
        self.widgets.append(widgets.Textarea(value=self.datastructure.widgetvals[2], continuous_update=False, layout=Layout(width = '600px', height = '100px')))
        self.widgets.append(widgets.Textarea(value=self.datastructure.widgetvals[3], continuous_update=False, layout=Layout(width = '600px', height = '100px')))
        self.widgets.append(widgets.Textarea(value=self.datastructure.widgetvals[4], continuous_update=False, layout=Layout(width = '600px', height = '100px')))
        self.widgets.append(widgets.Textarea(value=self.datastructure.widgetvals[5], continuous_update=False, layout=Layout(width = '600px', height = '100px')))
        self.widgets.append(widgets.Textarea(value=self.datastructure.widgetvals[6], continuous_update=False, layout=Layout(width = '600px', height = '100px')))
        self.widgets.append(widgets.Textarea(value=self.datastructure.widgetvals[7], continuous_update=False, layout=Layout(width = '600px', height = '100px')))
        self.widgets.append(widgets.Textarea(value=self.datastructure.widgetvals[8], continuous_update=False, layout=Layout(width = '600px', height = '100px')))
        self.widgets.append(widgets.Textarea(value=self.datastructure.widgetvals[9], continuous_update=False, layout=Layout(width = '600px', height = '100px')))
        
        for widget in self.widgets:
           widget.observe(self.widget_on_change, names='value')
        
        self.SaveData()
        
    def widget_on_change(self, change): #when widget values change (lose focus), write to file
        self.SaveData()
            
    def SaveData(self): 
        self.datastructure.widgetvals = [widget.value for widget in self.widgets]
        try:
            os.replace(self.file, self.tempfile)
        except FileNotFoundError: #after initial startup
            pass
        with open(self.file, 'wb') as outfile:
            pickle.dump(self.datastructure, outfile, pickle.HIGHEST_PROTOCOL)
            outfile.flush()
            os.fsync(outfile.fileno())
    
    def Save_show(self):
        dsply.display(self.SaveButton)
    
    def Save_pressed(self, btn):
        self.SaveData()
        
ans = Electronics_session4('Electronics_labs4.pkl')  

## Introduction


### Overview


You will investigate ways in which a voltage signal may be modulated. Each task requires two signals as inputs, in most cases one is generated by your function generator, the other is distributed around the lab and is available via a pair of wires at your workstation. The signal for experiments 1 will be available for the first hour, and the signal for experiments 2 and 3 is available thereafter. You will encounter low frequency (< 1Hz) signals; when displaying these the oscilloscope behaves somewhat differently to what you have seen previously, showing a 'rolling' display. You should ensure that you familiarise yourself with this display mode. Note that it is necessary to use the 'run/stop' button (upper right of the oscilloscope control panel) to freeze the display before using the measurement cursors in 'rolling' display mode as there is no trigger functionality.

The aims of this session are to gain experience of modulating and demodulating signals; to introduce the distributed signal, 'multiplier' PCB, and 'rolling' display mode of the oscilloscope; and to consolidate and refresh the experience with op-amps and low-pass filters from previous sessions. 



### Printed Circuit Boards



This week you are introduced to a new **multiplier** PCB, which is used in experiment 1. You do not need to understand the details of how this PCB operates. As with the amplifier and phase-shifter PCBs, the multiplier PCB requires a power supply in order to function: Connect the $-15V$, $0V$ and $+15V$ terminals of your PCBs to the $-V$, $0V$ and $+V$ outputs of your dual-rail power supply. 


The multiplier takes two inputs ($V_{in1}$, $V_{in2}$), and provides an ouput ($V_{out}$). Within practical limits (what are these likely to be?) the output is given by $V_{out} = V_{in1} \times V_{in2}$. 

You will also make use of the amplifier PCB, configured as a simple comparator as introduced in weeks 2 & 3, and the passive filter PCB introduced in week 1.

This session introduces the multiplier PCB and distributed signals, and recaps usage of the amplifier PCB and low-pass filters.



## Experiments

For the later parts of experiment 1 you will use a common signal distributed from the front of the lab. This is a 100 Hz sine-wave, but this signal will be replaced by a low-frequency signal for experiments 2 & 3 after the first hour of the lab session.



### Experiment 1 - Amplitude Modulation



**Amplitude Modulation** is the process whereby a signal is modulated onto a carrier wave by multiplying the two waveforms together, such that the amplitude of the carrier is multiplied by the amplitude of the signal at any instant in time. 

Connect a 1 kHz sine-wave from your signal-generator, your **'carrier wave'**, to input $V_{in1}$ of the multiplier PCB. 
You will need to set the DC-offset of the signal-generator output to be as close to zero as possible (why?).


### Tasks:


Connect each of the following signals to the $V_{in2}$ input, then observe and explain $V_{out}$ for:


- Various different DC voltages

In [7]:
# create and display text box for inputting answer
#widgets.Textarea(placeholder='Explain the output waveform:', layout=Layout(width = '600px', height = '100px'))
dsply.display(ans.widgets[0])

Textarea(value='Explain the form of the output signal:', continuous_update=False, layout=Layout(height='100px'…

- The 1 kHz sine wave from your signal generator (i.e. feed the same signal into both inputs of the multiplier). Reproduce the output using the waveform tool below; identify and explain its mathematical form

In [8]:
wfm1 = waveform(ans, ans.datastructure.wfmdata1)

In [9]:
wfm1.WaveGenButton()

Button(description='Generate Waveform', style=ButtonStyle())

Describe and explain the mathematical form of the signal:

In [10]:
# create and display text box for inputting answer
#widgets.Textarea(placeholder='Explain the output waveform:', layout=Layout(width = '600px', height = '100px'))
dsply.display(ans.widgets[1])

Textarea(value='Explain the form of the output signal:', continuous_update=False, layout=Layout(height='100px'…

- The 100 Hz waveform distributed throughout the lab. Reproduce and explain the output

In [11]:
wfm2 = waveform(ans, ans.datastructure.wfmdata2)

In [12]:
wfm2.WaveGenButton()

Button(description='Generate Waveform', style=ButtonStyle())

Describe and explain the mathematical form of the signal:

In [13]:
# create and display text box for inputting answer
#widgets.Textarea(placeholder='Explain the output waveform:', layout=Layout(width = '600px', height = '100px'))
dsply.display(ans.widgets[2])

Textarea(value='Explain the form of the output signal:', continuous_update=False, layout=Layout(height='100px'…

- What happens as you decrease the frequency of your carrier wave towards that of the signal?

In [14]:
# create and display text box for inputting answer
#widgets.Textarea(placeholder='Explain the output waveform:', layout=Layout(width = '600px', height = '100px'))
dsply.display(ans.widgets[3])

Textarea(value='Explain the form of the output signal:', continuous_update=False, layout=Layout(height='100px'…

At the end of this experiment you can disconnect the multiplier PCB as you will not need to use it again during this session. If you complete this experiment early, ask your demonstrator who may be able to provide you with a second signal generator to provide a 1 Hz sine-wave signal.



### Experiment 2 - Pulse Width Modulation


**Pulse Width Modulation** (PWM) is a simple modulation technique in which the mark-to-space ratio of a constant frequency two-level 'pulse-wave' is varied, changing the fraction of each cycle for which it is 'high' and 'low'. PWM is perhaps the simplest way in which a signal may be modulated, and has many uses; for example, in controlling the time-averaged power applied to devices such as motors and light sources. 


In this experiment you will produce a waveform with variable pulse-width. To achieve this, configure your signal generator to produce a sawtooth (triangular) waveform at the desired modulation frequency. You will again need to set the DC-offset of the signal-generator output to be as close to zero as possible (why?). 

Connect this sawtooth waveform to the inverting input to an op-amp configured as a **simple comparator** (no feedback components!). Apply a DC signal to the non-inverting input on your op-amp (what would happen if you switch the two inputs around?). A modulation frequency of around 1 kHz should work well for this experiment.


### Tasks:

- In your lab book, sketch the input (sawtooth) and output waveforms for several DC voltages.


- Explain the observed waveforms.

In [15]:
# create and display text box for inputting answer
#widgets.Textarea(placeholder='Explain the output waveform:', layout=Layout(width = '600px', height = '100px'))
dsply.display(ans.widgets[4])

Textarea(value='Explain the influence of the DC voltage on the comparator output:', continuous_update=False, l…

- In your lab book, sketch the low-frequency sine-wave signal distributed to your workstation. Annotate the sketch with signal parameters. You will need to use the 'Rolling' display mode on your oscilloscope and the 'run/stop' button to freeze the rolling display. It is important to ensure that you understand these - ask a demonstrator to explain if you don't understand.


- Apply the distributed low-frequency sine-wave at your circuit input in place of a DC signal.


- Describe and explain the observed output of the circuit below, and sketch it in your lab book.

In [16]:
# create and display text box for inputting answer
#widgets.Textarea(placeholder='Explain the output waveform:', layout=Layout(width = '600px', height = '100px'))
dsply.display(ans.widgets[5])

Textarea(value='Describe and explain the observed output:', continuous_update=False, layout=Layout(height='100…

You may wish to use the python file ``scope.py`` to capture images of this signal. Open an ``Anaconda command prompt`` and run this program (``python scope.py``).



Do **NOT** dismantle your working circuit - you will need it again shortly!



### Experiment 3 - Demodulation


The result of experiment 2 is that the low-frequency, continously-varying, input signal has been converted into a two-level pulse-width-modulated signal. You will now build an additional circuit to recover the original analogue signal from the PWM signal.


To recover the low-frequency signal, we need to filter the pulse train that is output from your comparator such that the average value encoded in the mark-to-space ratio is recovered whilst the high-frequency component, due to the pulses themselves, is rejected.



### Tasks:


- Specify a suitable corner frequency for a $RC$ low-pass filter given the ~1 kHz modulation frequency which you are using, and the condition that $f_{sig}$ should be less than $f_{mod}/100$. State the value and justify it below:

In [17]:
# create and display text box for inputting answer
#widgets.Textarea(placeholder='Explain the output waveform:', layout=Layout(width = '600px', height = '100px'))
dsply.display(ans.widgets[6])

Textarea(value='State and justify corner frequency value:', continuous_update=False, layout=Layout(height='100…

- Construct the *RC* filter and experimentally verify the corner frequency: produce a Bode plot for the gain only (not phase shift).

In [18]:
lp1 = bodedataset(ans, ans.datastructure.lpdata1, ['Frequency (Hz)', 'Frequency Error (Hz)', r'$V_{in}$ (mV)', r'$V_{out}$ (mV)'])

Button(description='Remove last line', style=ButtonStyle())

Button(description='Add line to table', style=ButtonStyle())

In [19]:
lp1.PlotButton()

Button(description='Generate Bode Plots', style=ButtonStyle())

- Apply your filter to the ouput of the circuit constructed in experiment 2. Sketch the demodulated signal in your lab book, and describe and explain it below:

In [20]:
dsply.display(ans.widgets[7])

Textarea(value='Describe and explain the demodulated signal obtained:', continuous_update=False, layout=Layout…

- What happens if you set your modulated frequency to be near, or below, the  corner frequency of the $RC$ filter?

In [21]:
dsply.display(ans.widgets[8])

Textarea(value='Explain what happens if the modulation frequency is too low:', continuous_update=False, layout…

### Extra task: 


- Remove the low-pass filter from the output of the comparator.


- Connect an LED between the comparator output and ground (on the terminal block).


- What behaviour do you observe from the LED? Why do you 'see' this?


In [22]:
dsply.display(ans.widgets[9])

Textarea(value='Explain the behaviour of the LED that you see:', continuous_update=False, layout=Layout(height…

In [23]:
#A save button: pretty pointless as the data is saved when the widget above loses focus, but this at least ensures that that soes happen.
ans.Save_show()

Button(description='Save all data', style=ButtonStyle())

Ensure that you press the 'save all data' button at the bottom of the notebook at the end of the session to ensure all data/inputs are saved, and exit the notebook by using 'File -> Close and Halt' from the Jupyter menu bar.


**Ensure that both students in your team have a working copy of the completed Jupyter Notebook, the generated '.pkl' datafile, and any sketches or notes required.**

In [24]:
# hides parts of the jupyter interface which the end user doesnt need to see and displays the page footer 
from IPython.display import HTML
HTML('''<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>''')